## Loading data

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-02 03:01:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 173.194.202.207, 173.194.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   172MB/s    in 0.4s    

2024-11-02 03:01:24 (172 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [4]:
from PIL import Image
import numpy as np
x_train = []
y_train = []
x_test = []
y_test = []

cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")
i = 0
while i < 800:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))
    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")

    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

while i < 1000:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))
    x_test.append(np.array(im_resized))
    y_test.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")

    x_test.append(np.array(im_resized))
    y_test.append(0)
  i += 1

## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [7]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

for layer in pretrained_model.layers:
  layer.trainable = False

final_model = tf.keras.models.Sequential()
final_model.add(pretrained_model)
final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(256, activation = "relu"))
final_model.add(tf.keras.layers.Dense(2, activation = "sigmoid"))

final_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                     loss = "binary_crossentropy",
                     metrics = ["accuracy"])

img_height, img_width = 150, 150
batch_size = 32
train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed = 123,
    label_mode = "categorical",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

validation = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    seed = 123,
    label_mode = "categorical",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

epochs = 3
history = final_model.fit(
    train,
    validation_data = validation,
    epochs = epochs
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Epoch 1/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 334s 5s/step - accuracy: 0.8684 - loss: 0.2758 - val_accuracy: 0.9570 - val_loss: 0.1034
Epoch 2/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 328s 4s/step - accuracy: 0.9780 - loss: 0.0619 - val_accuracy: 0.9610 - val_loss: 0.1124
Epoch 3/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 326s 5s/step - accuracy: 0.9889 - loss: 0.0365 - val_accuracy: 0.9430 - val_loss: 0.1484


In [8]:
from sre_constants import SUCCESS
probabilities = final_model.predict(x_test)
predictions = np.argmax(probabilities, axis=1)
successes = 0
fails = 0
for index in range(len(probabilities)):
  if predictions[index] == y_test[index]:
    successes += 1
  else:
    fails += 1

print("Test Data Accuracy: ", successes * 100 / (successes + fails), "%")

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_4_1/Cast:0", shape=(32, 150, 3), dtype=float32). Expected shape (None, 150, 150, 3), but input has incompatible shape (32, 150, 3)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', 'tf.Tensor(shape=(32, 150, 3), dtype=uint8)', '